Population predictor
=============

Using Deep Learning
------------

The following things are being done here.
1. Load dataset.npy file for the TPM count from quant.sf file. If not present:
    a. Read from the quant.sf files to a numpy array dataset.
    b. Save it using numpy.save file for faster access.
2. Perform Dimensionality reduction. Possible methods:
    a. PCA (Problems: Not for dataset with number of features larger than number of samples).
    b. TSNE (Slow)
    c. Encoder (Deep learning approach to encode into a lower dimensional form)
3. Apply the model to this reduced feature vector. Possible models:
    a. Fully connected neural network with softmax loss.
    b. Conv1D
    c. ...

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import StratifiedShuffleSplit
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
import yadlt

Read the input from the quant.sf file.

In [2]:
# Read the CSV file to map samples vs labels.
data_frame = pd.read_csv('p1_train.csv')
labels = list(set(data_frame['label']))

num_of_transcripts = 199324
num_of_samples = data_frame.shape[0]
num_labels = len(labels)

dataset = np.empty((num_of_samples, num_of_transcripts), dtype=np.float32)
sample_vs_label = np.empty(num_of_samples, dtype='int')
samples = np.empty(num_of_samples, dtype='<U9')

for index, row in data_frame.iterrows():
    if(index >= num_of_samples): break
    sample_vs_label[index] = labels.index(row['label'])
    samples[index] = row['accession']
print(sample_vs_label)

[0 2 0 0 2 3 4 3 2 3 3 1 1 3 3 4 4 0 0 3 2 4 3 3 1 0 0 1 2 1 0 2 3 1 1 4 4
 3 0 3 3 2 2 2 4 1 2 4 1 4 1 0 3 1 0 1 1 3 4 3 3 2 1 4 3 2 1 0 4 4 1 0 0 1
 0 1 4 0 0 0 0 3 0 3 0 2 2 4 2 0 0 3 2 4 4 3 0 4 1 1 3 4 0 3 2 1 1 4 1 1 2
 0 4 4 2 2 3 2 2 2 1 4 4 2 0 0 4 0 2 4 4 0 4 3 0 0 2 4 2 3 1 4 1 2 4 1 4 2
 1 4 3 0 0 1 4 3 2 4 0 2 3 0 4 4 3 2 1 1 2 2 2 4 4 4 3 1 4 4 0 0 3 2 2 0 2
 2 1 0 0 3 0 4 1 4 2 3 4 0 3 3 3 0 4 1 4 4 1 4 0 1 1 3 4 2 2 4 2 4 0 1 0 2
 1 2 4 3 1 0 3 3 2 1 3 2 0 2 1 1 4 1 2 1 2 2 2 2 2 2 2 0 1 3 3 1 4 0 0 1 0
 4 3 1 3 3 1 4 4 4 2 0 3 0 2 0 0 1 3 3 4 1 0 3 1 0 0 0 2 3 4 1 3 1 1 3 2 3
 0 3 3 1 4 4 4 3 3 1 2 4 2 1 4 3 2 1 1 1 3 1 2 3 0 4 2 2 0 4 2 3 3 4 3 2 1
 0 2 1 3 1 0 2 1 0 4 3 3 3 2 2 0 2 0 2 3 0 3 2 1 4 3 2 3 3 3 2 3 1 4 0 0]


Try and read from any saved 'dataset.npy' file. If present then load it to numpy array dataset, else just read it from all possible folders.

In [3]:
%%time
if(os.path.isfile('dataset.npy')):
    dataset = np.load('dataset.npy')
else:
    TRAIN_PATH = './train/'
    # Read the 'quant.sf' value here for each sample.
    for i in range(num_of_samples):
        if i%20==0: print(i)
        file_name = TRAIN_PATH + samples[i] + '/bias/quant.sf'
        quant_sf = np.genfromtxt(file_name, delimiter='\t', usecols=3, skip_header=True, dtype=np.float32)
        dataset[i] = quant_sf

    # Using https://i.stack.imgur.com/4d6yo.png to judge the best way to save the dataset as a npy file
    # for faster reloading of numpy array.
    np.save('dataset.npy', dataset)

CPU times: user 1.59 ms, sys: 216 ms, total: 218 ms
Wall time: 229 ms


# VarianceThreshold
Simple baseline approach to feature selection. It removes all features whose variance doesn’t meet some threshold. By default, it removes all zero-variance features, i.e. features that have the same value in all samples.
http://scikit-learn.org/stable/modules/feature_selection.html

In [4]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
dataset_reduced = sel.fit_transform(dataset)

dataset_reduced.shape

(369, 64843)

# Tree-based feature selection

Tree-based estimators (see the sklearn.tree module and forest of trees in the sklearn.ensemble module) can be used to compute feature importances, which in turn can be used to discard irrelevant features (when coupled with the sklearn.feature_selection.SelectFromModel meta-transformer):

In [4]:
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler = StandardScaler().fit(dataset)
dataset_reduced = scaler.transform(dataset)

clf = ExtraTreesClassifier()
clf = clf.fit(dataset_reduced[:int(dataset.size)], sample_vs_label[:int(dataset.size)])
clf.feature_importances_  

model = SelectFromModel(clf, prefit=True)
dataset_reduced = model.transform(dataset_reduced)
dataset_reduced.shape

(369, 1037)

# Applying PCA
## results = Test accuracy: 42.1%

Problem: Cannot reduce to dimensions greater than number of samples

In [6]:
num_of_transcripts = 300
pca = PCA(n_components=num_of_transcripts)
pca.fit(std_dataset)
dataset_reduced = pca.transform(std_dataset)

NameError: name 'std_dataset' is not defined

In [7]:
color_mapping = [sns.xkcd_rgb['blue'], sns.xkcd_rgb['lime'], sns.xkcd_rgb['ochre'], 
                 sns.xkcd_rgb['red'], sns.xkcd_rgb['green']]

colors = [color_mapping[x] for x in sample_vs_label]

In [ ]:
plt.scatter(dataset_reduced[:, 0], dataset_reduced[:, 10], c=colors)
plt.show()

# Applying TSNE
## (TODO)
Under construction

In [ ]:
model = TSNE(learning_rate=100, n_components=20, random_state=0, perplexity=10)
tsne5 = model.fit_transform(dataset)

# Applying Stacked Denoising Autoencoder 
## from yadlt 
https://github.com/blackecho/Deep-Learning-TensorFlow/

In [ ]:
sda = yadlt.SDAutoencoder(dims=[784, 400, 200, 80],
                    activations=["sigmoid", "sigmoid", "sigmoid"],
                    sess=sess,
                    noise=0.20,
                    loss="cross-entropy",
                    pretrain_lr=0.0001,
                    finetune_lr=0.0001)
sda.fit(dataset[:int(0.9*num_of_samples)])
dataset_reduced = sda.transform(dataset)

# Main modeling
## model data to machine learning tools
Split data to different datasets, namely, train, validation, test. Currently splitting in 0.8, 0.1, 0.1 ratios

def merge_datasets(dataset, train_size, valid_size, test_size):
  valid_dataset, valid_labels = dataset[:valid_size], sample_vs_label[:valid_size]
  train_dataset, train_labels = dataset[valid_size:valid_size+train_size], sample_vs_label[valid_size:valid_size+train_size]
  test_dataset, test_labels = dataset[valid_size+train_size:], sample_vs_label[valid_size+train_size:]
  return valid_dataset, valid_labels, train_dataset, train_labels, test_dataset, test_labels

train_size = int(0.8*num_of_samples)
valid_size = int(0.1*num_of_samples)
test_size = num_of_samples-valid_size-train_size

valid_dataset, valid_labels, train_dataset, train_labels, test_dataset, test_labels = merge_datasets(dataset_reduced, train_size, valid_size, test_size)
print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

train_dataset.shape[1]

Use One Hot encoding for the labels here

In [5]:
def reformat(labels):
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return labels
print(sample_vs_label[:5])
sample_vs_label = reformat(sample_vs_label)
print(sample_vs_label[:5])

[0 2 0 0 2]
[[ 1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]]


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [7]:
dataset_reduced.shape

(369, 1037)

# Neural Network
## FC Connected layer
Using a 2-layer fully connected neural network of 128, 64 relu neurons with batch size of 8

In [8]:
batch_size = 8
num_hidden_nodes = [128, 64]
num_of_transcripts = dataset_reduced.shape[1]
input_size = [num_of_transcripts]+num_hidden_nodes
test_size = 74
num_layers = len(num_hidden_nodes)

graph = tf.Graph()
with graph.as_default():
    # Input
    tf_train_dataset = tf.placeholder(tf.float32, shape=
                                     (batch_size, num_of_transcripts))
    tf_train_labels = tf.placeholder(tf.float32, shape=
                                    (batch_size, num_labels))
    tf_test_data = tf.placeholder(tf.float32, shape=
                                    (test_size, num_of_transcripts))
    print(tf_test_data.shape)
    global_step = tf.Variable(0)
    
    # Weights
    weights1 = []
    biases1 = []
    beta1 = []
    
    for i in range(num_layers):
        weights1.append(tf.Variable(tf.truncated_normal([input_size[i], num_hidden_nodes[i]],\
                                                        stddev=np.sqrt(2.0 / (input_size[i])))))
        biases1.append(tf.Variable(tf.zeros([num_hidden_nodes[i]])))
        beta1.append(1e-3)
    biases2 = tf.Variable(tf.zeros([num_labels]))
    weights2 = tf.Variable(tf.truncated_normal([input_size[-1], num_labels],\
                                               stddev=np.sqrt(2.0 / input_size[-1])))
    beta2 = 1e-3
    
    # Training Computation
    lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1[0]) + biases1[0])
    drop1 = tf.nn.dropout(lay1_train, 0.2)
    for i in range(1, num_layers):
        print(lay1_train.shape)
        lay1_train = tf.nn.relu(tf.matmul(drop1, weights1[i]) + biases1[i])
        drop1 = tf.nn.dropout(lay1_train, 0.2)
    logits = tf.matmul(drop1, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))\
                           + beta2 * tf.nn.l2_loss(weights2)
    for i in range(num_layers):
        loss += beta1[i] * tf.nn.l2_loss(weights1[i]) 
    
    # Optimizer
    learning_rate = tf.train.exponential_decay(1e-5, global_step, 1e6, 0.65, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
                           
    lay1_test = tf.nn.relu(tf.matmul(tf_test_data, weights1[0]) + biases1[0])
    for i in range(1, num_layers):
        lay1_test = tf.nn.relu(tf.matmul(lay1_test, weights1[i]) + biases1[i])
    logits_test = tf.matmul(lay1_test, weights2) + biases2
    test_prediction = tf.nn.softmax(logits_test)

(74, 1037)
(8, 128)


# Running the neural net
## Training for 18001 epochs

In [10]:
num_steps = 18000
result = np.zeros(5, dtype=np.float32)
i=0
accuracy_val = []
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialised')
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for train_index, test_index in sss.split(dataset_reduced, sample_vs_label):
        train_dataset, test_dataset = dataset_reduced[train_index], dataset_reduced[test_index]
        train_labels, test_labels = sample_vs_label[train_index], sample_vs_label[test_index]
        
        for step in range(num_steps):
            offset = (step * batch_size) % \
                    (train_labels.shape[0] - batch_size)

            batch_data = train_dataset[offset:offset+batch_size, :]
            batch_labels = train_labels[offset:offset+batch_size, :]

            feed_dict = { tf_test_data : test_dataset, tf_train_dataset : batch_data, \
                         tf_train_labels : batch_labels }
            _, l, predictions = session.run([optimizer, loss, \
                                             train_prediction], feed_dict=feed_dict)
#             if step%int(num_steps/2) == 0:
#                 print("Minibatch loss at step %d: %f" % (step, l))
#                 print("Minibatch accuracy: %.1f%% " % accuracy(predictions, batch_labels))
#                 print("Test accuracy: %.1f%% " % accuracy(test_prediction.eval(), test_labels))

        result[i] = accuracy(test_prediction.eval(feed_dict={tf_test_data : test_dataset}), test_labels)
        print("Test accuracy: %.1f%%" % result[i])
        i+=1
    print(np.mean(result))


Initialised
Test accuracy: 78.4%
Test accuracy: 89.2%
Test accuracy: 91.9%
Test accuracy: 98.6%
Test accuracy: 97.3%
91.0811
